In [1]:
import pandas as pd
from datetime import datetime
from collections import deque

def read_log(path):

    cams = dict() # キー=カメラ, 値=日時リスト

    with open(path, 'r', encoding='shift_jis', errors='ignore') as f:
        read_state = 0
        que = deque(maxlen=5)
        while True:
            line = f.readline()
            if not line:
                break

            que.append(line) # いったん流れた日時行を取得するため直近5行をキューに保持

            if read_state == 0:   # 開始行の探索中
                if line.startswith('サブリスト: 動体検知開始'):
                    line = que[-4] # 日時行
                    pos = line.find(' ')
                    dt = datetime.strptime(line[pos+1:].strip(), '%Y-%m-%d %H:%M:%S')
                    read_state = 1

            elif read_state == 1: # カメラ行の探索中
                if line.startswith('カメラ No.:'):
                    cam = line.split(':')[1].strip()
                    if cam not in cams:
                        cams[cam] = []
                    cams[cam].append(dt)
                    read_state = 0

    return cams

cams = read_log('logBack.txt')
print('read_log end.')

# 結果をカメラ毎にCSV出力
for cam, dts in cams.items():
    df = pd.DataFrame({'Date':dts})
    df['cnt'] = 1
    df = df.set_index('Date')
    df = df.resample('H').sum()
    df.to_csv(f'{cam}.csv')

read_log end.


In [ ]:
print('test')